In [1]:
from crypto_pandas import CCXTPandasExchange
import ccxt
import pandas as pd

In [2]:
# Setup exchanges
binance = CCXTPandasExchange(exchange=ccxt.binance())
bybit = CCXTPandasExchange(exchange=ccxt.bybit())
kucoin = CCXTPandasExchange(ccxt.kucoinfutures())

In [3]:
# Fetch bids and asks
binance_ba = binance.fetch_bids_asks()
bybit_ba = bybit.fetch_bids_asks()
kucoin_ba = kucoin.fetch_bids_asks()

# Combine and display
data = pd.concat([
    binance_ba,
    bybit_ba,
    kucoin_ba,
])

print(data.dropna(axis=1))

              symbol  bidVolume   askVolume       exchange
0            ETH/BTC    18.2136     37.1072        binance
1            LTC/BTC     6.9370    279.8960        binance
2            BNB/BTC     3.5580      2.3210        binance
3            NEO/BTC    25.3100    813.7900        binance
4           QTUM/ETH  4529.4000     42.4000        binance
..               ...        ...         ...            ...
453     AR/USDT:USDT   133.0000   1593.0000  kucoinfutures
454     B3/USDT:USDT  2016.0000   1086.0000  kucoinfutures
455   PNUT/USDT:USDT  5235.0000  11395.0000  kucoinfutures
456  SYRUP/USDT:USDT    85.0000    477.0000  kucoinfutures
457   HIGH/USDT:USDT    77.0000     77.0000  kucoinfutures

[4254 rows x 4 columns]


In [4]:
# Find all combinations and calculate spread
arbitrage = data[["exchange", "symbol", "bid"]].merge(data[["exchange", "symbol", "ask"]], on="symbol", suffixes=("Short", "Long"))
arbitrage["spread"] = arbitrage["bid"] - arbitrage["ask"]
arbitrage["spreadRelative"] = arbitrage["spread"]/arbitrage[["bid", "ask"]].mean(axis=1)

# Filter for different exchanges for buy/sell
arbitrage = arbitrage.query("exchangeShort != exchangeLong").sort_values("spreadRelative", ascending=False, ignore_index=True)
print(arbitrage.head(10).to_markdown(index=False))

| exchangeShort   | symbol           |      bid | exchangeLong   |      ask |   spread |   spreadRelative |
|:----------------|:-----------------|---------:|:---------------|---------:|---------:|-----------------:|
| bybit           | ALU/USDT:USDT    | 0.006387 | kucoinfutures  | 0.00636  |  2.7e-05 |       0.00423629 |
| bybit           | TSTBSC/USDT:USDT | 0.04625  | kucoinfutures  | 0.04608  |  0.00017 |       0.00368244 |
| bybit           | XEM/USDT:USDT    | 0.003257 | kucoinfutures  | 0.003246 |  1.1e-05 |       0.00338305 |
| bybit           | GORK/USDT:USDT   | 0.0079   | kucoinfutures  | 0.007874 |  2.6e-05 |       0.00329656 |
| bybit           | COTI/USDT:USDT   | 0.06429  | kucoinfutures  | 0.06414  |  0.00015 |       0.0023359  |
| bybit           | AI/USDT:USDT     | 0.1437   | kucoinfutures  | 0.1434   |  0.0003  |       0.00208986 |
| kucoinfutures   | SONIC/USDT:USDT  | 0.2472   | bybit          | 0.2467   |  0.0005  |       0.0020247  |
| bybit           | ROAM/USD